In [1]:
from pdp_utils import *
from tqdm import tqdm
import time
import random
import numpy as np
n = 10000

def oneopt_operator_nborhood(cur_sol):
    nbors = []
    for i in range(len(cur_sol)):
        currentlist = []
        for j in range(len(cur_sol)):
            insertable = cur_sol.copy()
            item_to_insert = insertable[i]
            insertable = insertable[:i] + insertable[i+1:]
            insertable.insert(j, item_to_insert)
            currentlist.append(insertable)
        nbors = nbors + currentlist
    return nbors

def twoopt_operator_nborhood(cur_sol):
    
    nbors = []
    for i in range(len(cur_sol)):
        for j in range(len(cur_sol)):
            insertable = cur_sol.copy()
            i_val = insertable[i]
            insertable[i] = insertable[j]
            insertable[j] = i_val
            nbors.append(insertable)
    return nbors

def threeopt_operator_nborhood(cur_sol):
    nbors = []
    for i in range(len(cur_sol)):
        for j in range(len(cur_sol)):
            for k in range(len(cur_sol)):
                insertable = cur_sol.copy()
                i_val = insertable[i]
                j_val = insertable[j]
                insertable[i] = insertable[k]
                insertable[j] = i_val
                insertable[k] = j_val
                nbors.append(insertable)
    return nbors

def solution_sanity(sol):
    """Ensures that any given solution both picks up and delivers its assigned calls"""
    currentcalls = []
    for i in range(len(sol)):
        if sol[i] == 0: 
            if len(currentcalls) > 0:
                return False
        else:
            if sol[i] in currentcalls:
                currentcalls.remove(sol[i])
            else:
                currentcalls.append(sol[i])
    return True

def rejected_calls_cost(sol, prob):
    total = 0
    sol.reverse()
    for call in sol:
        if call == 0:
            return total
        total += prob["Cargo"][call - 1][3]
    raise ValueError("Bad solution")
        
    

In [22]:
def localsearch(init_sol, operator, prob):
    best_sol = init_sol
    best_sol_cost = cost_function(best_sol, prob)
    for i in tqdm(range(n)):
        best_nbor = None
        best_nbor_cost = float('inf')
        nbors = operator(best_sol)
        nbors = [nbor for nbor in nbors if solution_sanity(nbor)]
        for nbor in nbors:
            feasibility, c = feasibility_check(nbor, prob)
            if feasibility:
                nbor_cost = cost_function(nbor, prob) + rejected_calls_cost(nbor, prob)
                if nbor_cost <= best_nbor_cost:
                    best_nbor_cost = nbor_cost
                    best_nbor = nbor
        
        if best_nbor is not None and best_sol_cost > best_nbor_cost:
            best_sol = best_nbor
            best_sol_cost = best_nbor_cost
    return best_sol, best_sol_cost

def annealing(init_sol, operator, prob):
    best_sol = init_sol
    best_sol_cost = cost_function(best_sol, prob)
    delta_es = []
    
    for i in tqdm(range(100)):
        best_nbor = None
        best_nbor_cost = float('inf')
        nbors = operator(best_sol)
        nbors = [nbor for nbor in nbors if solution_sanity(nbor)]
        for nbor in nbors:
            feasibility, c = feasibility_check(nbor, prob)
            if feasibility:
                nbor_cost = cost_function(nbor, prob) + rejected_calls_cost(nbor, prob)
                if nbor_cost < best_nbor_cost:
                    best_nbor_cost = nbor_cost
                    best_nbor = nbor
                else:
                    randval = random.random()
                    if randval < 0.8:
                        best_nbor_cost = nbor_cost
                        best_nbor = nbor
        
        if best_nbor is not None:
            delta_es.append(best_sol_cost - best_nbor_cost)
            if best_sol_cost > best_nbor_cost:
                best_sol = best_nbor
                best_sol_cost = best_nbor_cost
    T = (sum(delta_es) / len(delta_es)) / np.log(0.8)
    alpha = np.sqrt(0.1 / T)
    
    for i in tqdm(range(n - 100)):
        best_nbor = None
        best_nbor_cost = float('inf')
        nbors = operator(best_sol)
        nbors = [nbor for nbor in nbors if solution_sanity(nbor)]
        for nbor in nbors:
            feasibility, c = feasibility_check(nbor, prob)
            if feasibility:
                nbor_cost = cost_function(nbor, prob) + rejected_calls_cost(nbor, prob)
                if nbor_cost < best_nbor_cost:
                    best_nbor_cost = nbor_cost
                    best_nbor = nbor
                else:
                    delta_e = nbor_cost - best_nbor_cost
                    randval = random.random()
                    if randval < np.exp(-delta_e / T):
                        best_nbor_cost = nbor_cost
                        best_nbor = nbor
        
        if best_nbor is not None:
            delta_es.append(best_sol_cost - best_nbor_cost)
            if best_sol_cost > best_nbor_cost:
                best_sol = best_nbor
                best_sol_cost = best_nbor_cost
        T = T * alpha

        
    return best_sol, best_sol_cost

In [ ]:
def run_problem(problem, initial_solution):
    methods = [localsearch, annealing]
    operators = [oneopt_operator_nborhood, twoopt_operator_nborhood, threeopt_operator_nborhood]
    prob = load_problem('./Call_7_Vehicle_3.txt')
    mtod = []
    for method in methods
        oprtr = []
        for operator in operators:
            best_sol, best_sol_cost = method(initial_solution, operator, prob)
            oprtr.append(best_sol, best_sol_cost)
        mtod.append(oprtr)

# C7V3

In [20]:
initial_solution = [7, 7, 0, 3, 3, 0, 1, 5, 5, 1, 0, 6, 4, 2]
prob = load_problem('./Call_7_Vehicle_3.txt')
for method in methods
    for operator in operators:
        best_sol, best_sol_cost = method(initial_solution, operator, prob)
        print(best_sol, best_sol_cost)

  1%|          | 81/9900 [00:02<06:06, 26.79it/s]<ipython-input-18-3f66f487a469>:68: RuntimeWarning: overflow encountered in double_scalars
  if randval < np.exp(-delta_e / T):
  1%|          | 87/9900 [00:02<07:13, 22.64it/s]<ipython-input-18-3f66f487a469>:68: RuntimeWarning: invalid value encountered in double_scalars
  if randval < np.exp(-delta_e / T):
<ipython-input-18-3f66f487a469>:68: RuntimeWarning: divide by zero encountered in double_scalars
  if randval < np.exp(-delta_e / T):
100%|██████████| 9900/9900 [05:36<00:00, 29.46it/s]

[7, 7, 0, 3, 3, 0, 1, 5, 5, 1, 0, 6, 4, 2] 1234724.5


In [21]:
solution_sanity([7, 7, 0, 3, 3, 0, 1, 5, 5, 1, 0, 6, 4, 2])

True